## Тестовое задание для проекта «Parler-TTS для русского языка»

### План оценки модели

Нужно выбрать тексты, которые позволят оценить основные характеристики синтеза речи. Проверить синтез эмоционально окрашенных фраз, чисел, дат и акронимов, чтобы оценить способность модели корректно обрабатывать эти элементы.

Можно сравнить ... (декомпозирование понятия "оценка синтеза речи")
- Насколько голос звучит естественно (не как робот)
- Правильное использование интонации для разных типов предложений
- Чёткость и корректность произнесения звуков
- Пол и высота звука говорящего
- Насколько синтезированный звук соответствует входному тексту
- Передача эмоций в голосе
- Скорость говорения

Обозначу особенности датасета и тестируемой модели.
Датасет содержит аудиозаписи только женского голоса и на английском языке, поэтому исключаем некоторые вышеперечисленные критерии оценки.
Остаются критерии:
- Насколько голос звучит естественно
- Чёткость и корректность произнесения звуков
- Насколько синтезированный звук соответствует входному тексту
- Передача эмоций в голосе
- Сходство с целевым голосом Jenny

Для тестирования использовались данные https://huggingface.co/datasets/ylacombe/jenny-tts-6h

### Выбор метрик для оценки качества
- PESQ (Perceptual Evaluation of Speech Quality)
- Mel Cepstral Distortion (MCD)
- Signal-to-noise ratio (SNR)
- WER (Word Error Rate)
Далее будет краткое сравнение метрик

Получить субъективную оценку слушателя (то есть меня :)) и провести сравнение между синтезированным и эталонным аудио.

### Почему эти метрики?

1. PESQ
- Измеряет восприятие качества речи на слух, сравнивая синтезированное аудио с эталонным. Хорошо отражает воспринимаемую "чистоту" речи, наличие шумов и артефактов.
Ограничение - ориентирована больше на технические искажения, чем на естественность или эмоциональность.

2. Mel
- Измеряет расстояние между спектрограммами эталонного и синтезированного звука. Хорошо подходит для технической оценки соответствия тембра и артикуляции. Чувствительна к деталям синтеза (например, мелодия, тембр, частотные характеристики). Это сугубо техническая метрика.

3. SNR
- В контексте TTS SNR измеряет процент нежелательного шума в аудиопотоке относительно распознаваемой речи. Это влияет на способность слышать и понимать речь при наличии фонового шума.

4. WER
- Измеряет расхождения между текстом, распознанным из синтезированной речи, и исходным текстом. Простой способ проверить, насколько результат понятен ASR-системам.

На данный момент нет универсальной метрики, которая могла бы заменить все перечисленные. Поэтому следует провести сравнение по всем.
Информация взята с Torch Metrics

### Генерация аудио с помощью модели

Промпт был сгенерирован с помощью ChatGPT следующим образом:
"Ты - известный писатель на английском языке. Напиши небольшой отрывок прозаического текста с обязательным использованием вопросительного предложения, восклицательного предложения, многоточия, число и дата от лица женщины."

In [22]:
import torch
import os
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-jenny-30H").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-jenny-30H")
# Проверим, сможет ли прочитать числительные в таком формате.
prompt = "April 14th, 2025. It was almost 8:00 PM, and the room felt heavier than usual. My mind replayed his words: “Sometimes, love isn’t enough.” How cruel! How could he write that and still expect me to understand? Outside, a car horn shattered the stillness, followed by an impatient shout."
description = "Jenny speaks at an average pace with an animated delivery in a very confined sounding environment with clear audio quality."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Создание директории для сохранения результатов
output_dir = "generated_outputs"
os.makedirs(output_dir, exist_ok=True)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()

output_path = os.path.join(output_dir, "parler_tts_out_01.wav")
sf.write(output_path, audio_arr, model.config.sampling_rate)

  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length"

In [23]:
import torch
import os
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-jenny-30H").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-jenny-30H")

prompt = "April fourteenth twenty twenty five. It was almost eight PM, and the room felt heavier than usual. My mind replayed his words: “Sometimes, love isn’t enough.” How cruel! How could he write that and still expect me to understand? Outside, a car horn shattered the stillness, followed by an impatient shout."

# Создание директории для сохранения результатов
output_dir = "generated_outputs"
os.makedirs(output_dir, exist_ok=True)

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()

output_path = os.path.join(output_dir, "parler_tts_out_02.wav")
sf.write(output_path, audio_arr, model.config.sampling_rate)

  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length"

### Наблюдения после генерации

По наблюдениям, при первой генерации аудиоролик не содержал произношения даты и времени. Однако, все знаки препинания интонационно соблюдены, что очень даже неплохо. Там, где стоит многоточие, чувствуется длинная пауза. Произношение слов чёткое и корректное, голос максимально похож на оригинальный Jenny.
Вторая генерация, когда все числительные были переписаны словами, дала более качественный результат, потому что все слова были произнесены.
Сильной роботизации не чувствуется в голосе.

### Оценка синтеза

In [21]:
from datasets import load_dataset
import librosa
from jiwer import wer
import numpy as np
from tqdm import tqdm
import logging

# Логирование
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

def load_model_and_tokenizer(model_name):
    """Загрузка модели и токенизатора с обработкой ошибок."""
    try:
        model = ParlerTTSForConditionalGeneration.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        return model, tokenizer
    except Exception as e:
        logger.error(f"Error loading model and tokenizer: {e}")
        raise

def calculate_snr(reference, synthesized, sr):
    """
    Расчет отношения сигнал/шум (SNR) между эталонным и синтезированным аудио.
    """
    try:
        min_length = min(len(reference), len(synthesized))
        reference = reference[:min_length]
        synthesized = synthesized[:min_length]
        
        signal_power = np.mean(reference ** 2)
        
        noise = reference - synthesized
        noise_power = np.mean(noise ** 2)
        
        if noise_power == 0:
            return float('inf')
        
        snr = 10 * np.log10(signal_power / noise_power)
        return snr
    except Exception as e:
        logger.error(f"Error computing SNR: {e}")
        return float('-inf')

def generate_audio_and_text(model, tokenizer, description, text):
    try:
        # Токенизация описания и текста
        tokens_description = tokenizer(
            description,
            return_tensors="pt",
            padding=True,
            truncation=True
        )
        
        tokens_text = tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True
        )
        
        # Перемещение тензоров на нужное устройство
        input_ids = tokens_description.input_ids.to(device)
        prompt_input_ids = tokens_text.input_ids.to(device)
        
        # Генерация аудио
        with torch.no_grad():
            generation = model.generate(
                input_ids=input_ids,
                prompt_input_ids=prompt_input_ids,
                max_length=1000
            )
            
            # Генерация текста (используем те же входные данные)
            text_output = tokenizer.batch_decode(prompt_input_ids, skip_special_tokens=True)[0]
        
        return generation.cpu().numpy().squeeze(), text_output
    except Exception as e:
        logger.error(f"Error in generation: {e}")
        raise

def evaluate_metrics(dataset, model, tokenizer, num_samples=10):  # Изменено на 10 сэмплов, изначально стояли все
    snr_scores = []
    wer_scores = []
    failed_samples = 0
    
    total_samples = min(num_samples, len(dataset))
    logger.info(f"Processing {total_samples} samples")
    
    default_description = "Jenny speaks at an average pace with an animated delivery in a very confined sounding environment with clear audio quality."

    for idx in tqdm(range(total_samples), desc="Processing samples"):
        try:
            sample = dataset[idx]

            required_keys = ["transcription", "audio", "transcription_normalised"]
            if not all(key in sample for key in required_keys):
                logger.warning(f"Sample {idx} missing required keys")
                failed_samples += 1
                continue

            # Нужен текстовый промпт и аудиофайл
            prompt = sample["transcription"]
            audio_info = sample["audio"]
            ref_audio = audio_info["array"]
            sr = audio_info["sampling_rate"]

            # Проверка валидности аудио
            if len(ref_audio) == 0:
                logger.warning(f"Empty reference audio in sample {idx}")
                failed_samples += 1
                continue

            try:
                gen_audio, gen_text = generate_audio_and_text(model, tokenizer, default_description, prompt)
            except Exception as e:
                logger.error(f"Failed to generate audio/text for sample {idx}: {e}")
                failed_samples += 1
                continue

            # Преобразование эталонного аудио к нужной частоте дискретизации для метрики
            if sr != model.config.sampling_rate:
                ref_audio = librosa.resample(ref_audio, orig_sr=sr, target_sr=model.config.sampling_rate)

            snr = calculate_snr(ref_audio, gen_audio, model.config.sampling_rate)
            if snr != float('-inf'):
                snr_scores.append(snr)

            ref_text = sample["transcription_normalised"]
            try:
                wer_score = wer(ref_text, gen_text)
                wer_scores.append(wer_score)
                logger.debug(f"Sample {idx} - Reference: '{ref_text}', Generated: '{gen_text}'")
            except Exception as e:
                logger.error(f"Error calculating WER for sample {idx}: {e}")

            # Логирование промежуточных результатов
            if (idx + 1) % 2 == 0 and snr_scores and wer_scores:  # Каждые 2 сэмпла
                logger.info(f"Processed {idx + 1} samples. Current averages - SNR: {np.mean(snr_scores):.4f} dB, WER: {np.mean(wer_scores):.4f}")

        except Exception as e:
            logger.error(f"Error processing sample {idx}: {e}")
            failed_samples += 1
            continue

    if not snr_scores and not wer_scores:
        raise ValueError("No valid metrics were computed")
    
    results = {}
    if snr_scores:
        results["SNR"] = np.mean(snr_scores)
    if wer_scores:
        results["WER"] = np.mean(wer_scores)
    
    results.update({
        "processed_samples": total_samples - failed_samples,
        "failed_samples": failed_samples
    })

    return results

def main():
    try:
        model_name = "parler-tts/parler-tts-mini-jenny-30H"
        model, tokenizer = load_model_and_tokenizer(model_name)

        # Загрузка датасета jenny-tts-6h
        dataset = load_dataset("ylacombe/jenny-tts-6h", split="train")

        metrics = evaluate_metrics(dataset, model, tokenizer)
        
        logger.info("Финальные метрики качества:")
        for metric, value in metrics.items():
            logger.info(f"{metric}: {value:.4f}" if isinstance(value, float) else f"{metric}: {value}")
            
    except Exception as e:
        logger.error(f"Fatal error: {e}")
        raise

if __name__ == "__main__":
    main()

INFO:__main__:Using device: cpu
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_sto

### Описание полученных результатов

По итогам тестирования (к сожалению, на 10 записях иначе не успело бы выполниться до дедлайна) были получены значения двух метрик:

|**SNR**   |**WER**   |
|----------|----------|
|-2,8605   |0,1340    |

Они являются вполне нормальными для данной модели, учитывая особенности тренировочного датасета.
Было бы, конечно, гораздо интереснее тестировать модель, которая обучалась на фразах разных дикторов (разного пола, акцента...)

Оценивая субъективные метрики, можно сказать, что текст генерируется довольно четко, хорошо воспринимается человечиским слухом, учитываются паузы и эмоциональная окраска речи на вполне достойном уровне, голос не кажется роботизированным.

### Небольшой комментарий
При попытке учесть метрику PESQ возникли проблемы с установкой библиотеки на Windows...поэтому она не была учтена в работе
Не хватило времени разобраться глубже в проблеме и задаче, но тестировать очень понравилось :)